In [1]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold

import tensorflow as tf

pd.options.display.max_columns = 100

%matplotlib inline

radiant_cols = ['hero_' + str(i) for i in range(5)]
dire_cols = ['hero_' + str(i) for i in range(5, 10)]

unique_roles = [
    'Carry',
    'Disabler',
    'Durable',
    'Escape',
    'Initiator',
    'Jungler',
    'Nuker',
    'Pusher',
    'Support'
]

unique_primary_attrs = [
    'agi',
    'int',
    'str'
]

feature_names = [
    'agi_gain',
    'attack_range',
    'attack_rate',
    'base_agi',
    'base_armor',
    'base_attack_max',
    'base_attack_min',
    'base_health_regen',
    'base_int',
    'base_mr',
    'base_str',
    'int_gain',
    'legs',
    'move_speed',
    'pro_ban',
    'pro_pick',
    'pro_win',
    'projectile_speed',
    'str_gain',
    'turn_rate',
    'no_agi',
    'no_int',
    'no_str',
    'no_melees',
    'no_Carry',
    'no_Disabler',
    'no_Durable',
    'no_Escape',
    'no_Initiator',
    'no_Jungler',
    'no_Nuker',
    'no_Pusher',
    'no_Support',
]

def load_data():
    df = pd.read_csv('../data/matches_data.csv')
    radiant_features = pd.read_csv('../data/radiant_features.csv')
    dire_features = pd.read_csv('../data/dire_features.csv')
    
    assert np.all(radiant_features.columns == dire_features.columns)
    assert df.shape[0] == radiant_features.shape[0]
    assert df.shape[0] == dire_features.shape[0]
    assert len(feature_names) == radiant_features.shape[1]
    assert len(feature_names) == dire_features.shape[1]
    
    radiant_features = radiant_features.rename({col: 'radiant_' + col for col in feature_names}, axis=1)
    dire_features = dire_features.rename({col: 'dire_' + col for col in feature_names}, axis=1)
    df = pd.concat([df[['match_id', 'radiant_win']], radiant_features, dire_features], axis=1)
    return df

/home/hardian_lawi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df = load_data()
df.head()

,match_id,radiant_win,radiant_agi_gain,radiant_attack_range,radiant_attack_rate,radiant_base_agi,radiant_base_armor,radiant_base_attack_max,radiant_base_attack_min,radiant_base_health_regen,radiant_base_int,radiant_base_mr,radiant_base_str,radiant_int_gain,radiant_legs,radiant_move_speed,radiant_pro_ban,radiant_pro_pick,radiant_pro_win,radiant_projectile_speed,radiant_str_gain,radiant_turn_rate,radiant_no_agi,radiant_no_int,radiant_no_str,radiant_no_melees,radiant_no_Carry,radiant_no_Disabler,radiant_no_Durable,radiant_no_Escape,radiant_no_Initiator,radiant_no_Jungler,radiant_no_Nuker,radiant_no_Pusher,radiant_no_Support,dire_agi_gain,dire_attack_range,dire_attack_rate,dire_base_agi,dire_base_armor,dire_base_attack_max,dire_base_attack_min,dire_base_health_regen,dire_base_int,dire_base_mr,dire_base_str,dire_int_gain,dire_legs,dire_move_speed,dire_pro_ban,dire_pro_pick,dire_pro_win,dire_projectile_speed,dire_str_gain,dire_turn_rate,dire_no_agi,dire_no_int,dire_no_str,dire_no_melees,dire_no_Carry,dire_no_Disabler,dire_no_Durable,dire_no_Escape,dire_no_Initiator,dire_no_Jungler,dire_no_Nuker,dire_no_Pusher,dire_no_Support
0,4154000815,False,7.7,1230.0,8.8,74.0,-2.0,219.0,177.0,9.25,96.0,125.0,116.0,11.8,10.0,1435.0,295.0,211.0,108.0,4000.0,12.40,3.20,1.0,2.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,0.0,4.0,1.0,2.0,10.50,1900.0,8.5,96.0,-5.0,144.0,108.0,9.75,87.0,110.0,101.0,9.10,8.0,1445.0,219.0,231.0,125.0,5000.0,12.5,4.10,3.0,1.0,1.0,2.0,3.0,4.0,2.0,3.0,2.0,0.0,3.0,1.0,2.0
1,4152678603,True,11.3,2190.0,8.6,90.0,0.0,157.0,120.0,7.50,87.0,125.0,95.0,12.3,10.0,1485.0,238.0,315.0,160.0,4700.0,10.35,3.40,2.0,2.0,1.0,1.0,4.0,3.0,1.0,2.0,3.0,0.0,5.0,1.0,2.0,11.70,1450.0,8.2,98.0,2.0,180.0,147.0,11.00,92.0,125.0,114.0,10.40,10.0,1485.0,447.0,311.0,151.0,3700.0,12.0,2.70,2.0,2.0,1.0,3.0,3.0,4.0,2.0,1.0,3.0,2.0,3.0,1.0,2.0
2,4152494309,False,9.8,1200.0,8.6,91.0,5.0,177.0,147.0,8.00,88.0,125.0,106.0,9.3,10.0,1485.0,148.0,103.0,56.0,3600.0,12.40,3.35,2.0,1.0,2.0,4.0,4.0,5.0,3.0,2.0,5.0,0.0,5.0,1.0,2.0,9.35,1150.0,8.5,76.0,0.0,180.0,147.0,8.75,78.0,125.0,113.0,10.55,10.0,1490.0,196.0,313.0,159.0,2700.0,13.6,3.30,1.0,1.0,3.0,4.0,1.0,3.0,2.0,2.0,2.0,0.0,4.0,0.0,3.0
3,4152425012,False,8.7,2000.0,8.4,92.0,-2.0,175.0,143.0,8.00,102.0,125.0,102.0,11.4,10.0,1470.0,NaN,NaN,NaN,4175.0,12.20,3.20,1.0,3.0,1.0,2.0,3.0,5.0,1.0,3.0,2.0,0.0,5.0,0.0,3.0,10.60,1575.0,8.3,89.0,-4.0,139.0,101.0,7.75,91.0,125.0,114.0,9.10,8.0,1430.0,385.0,377.0,199.0,4000.0,13.0,3.45,2.0,1.0,2.0,2.0,2.0,5.0,2.0,3.0,3.0,0.0,5.0,1.0,2.0
4,4152417604,False,6.4,1680.0,8.5,80.0,-3.0,175.0,138.0,10.25,91.0,125.0,105.0,12.4,10.0,1530.0,347.0,374.0,195.0,2700.0,12.20,3.00,1.0,2.0,2.0,3.0,2.0,3.0,1.0,2.0,2.0,1.0,4.0,2.0,2.0,10.00,1075.0,8.1,96.0,1.0,176.0,148.0,8.50,86.0,125.0,113.0,9.50,10.0,1465.0,310.0,317.0,136.0,3900.0,13.7,2.90,3.0,1.0,1.0,4.0,3.0,5.0,2.0,2.0,3.0,2.0,4.0,0.0,1.0
